In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("mylogistic").getOrCreate()

In [3]:
from pyspark.ml.classification import LogisticRegression

In [4]:
my_data = spark.read.csv('/FileStore/tables/titanic.csv', header = True, inferSchema = True)

In [5]:
my_data.show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
PassengerId|Survived|Pclass| Name| Sex| Age|SibSp|Parch| Ticket| Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
 1| 0| 3|Braund, Mr. Owen ...| male|22.0| 1| 0| A/5 21171| 7.25| null| S|
 2| 1| 1|Cumings, Mrs. Joh...|female|38.0| 1| 0| PC 17599|71.2833| C85| C|
 3| 1| 3|Heikkinen, Miss. ...|female|26.0| 0| 0|STON/O2. 3101282| 7.925| null| S|
 4| 1| 1|Futrelle, Mrs. Ja...|female|35.0| 1| 0| 113803| 53.1| C123| S|
 5| 0| 3|Allen, Mr. Willia...| male|35.0| 0| 0| 373450| 8.05| null| S|
 6| 0| 3| Moran, Mr. James| male|null| 0| 0| 330877| 8.4583| null| Q|
 7| 0| 1|McCarthy, Mr. Tim...| male|54.0| 0| 0| 17463|51.8625| E46| S|
 8| 0| 3|Palsson, Master. ...| male| 2.0| 3| 1| 349909| 21.075| null| S|
 9| 1| 3|Johnson, Mrs. Osc...|female|27.0| 0| 2| 347742|11.1333| null| S|
 10| 1| 2|Nasser, Mrs. Nich...|female|14.0| 1| 0| 237736|30.0708| null| C|
 11| 1| 3|Sandstrom, Miss. ...|female| 4.0| 1| 1| PP 9549| 16.7| G6| S|
 12| 1| 1|Bonnell, Miss. El...|female|58.0| 0| 0| 113783| 26.55| C103| S|
 13| 0| 3|Saundercock, Mr. ...| male|20.0| 0| 0| A/5. 2151| 8.05| null| S|
 14| 0| 3|Andersson, Mr. An...| male|39.0| 1| 5| 347082| 31.275| null| S|
 15| 0| 3|Vestrom, Miss. Hu...|female|14.0| 0| 0| 350406| 7.8542| null| S|
 16| 1| 2|Hewlett, Mrs. (Ma...|female|55.0| 0| 0| 248706| 16.0| null| S|
 17| 0| 3|Rice, Master. Eugene| male| 2.0| 4| 1| 382652| 29.125| null| Q|
 18| 1| 2|Williams, Mr. Cha...| male|null| 0| 0| 244373| 13.0| null| S|
 19| 0| 3|Vander Planke, Mr...|female|31.0| 1| 0| 345763| 18.0| null| S|
 20| 1| 3|Masselmani, Mrs. ...|female|null| 0| 0| 2649| 7.225| null| C|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
only showing top 20 rows

In [6]:
my_data.printSchema()

root
-- PassengerId: integer (nullable = true)
-- Survived: integer (nullable = true)
-- Pclass: integer (nullable = true)
-- Name: string (nullable = true)
-- Sex: string (nullable = true)
-- Age: double (nullable = true)
-- SibSp: integer (nullable = true)
-- Parch: integer (nullable = true)
-- Ticket: string (nullable = true)
-- Fare: double (nullable = true)
-- Cabin: string (nullable = true)
-- Embarked: string (nullable = true)

In [7]:
my_data.columns

Out[ 18 ]: 
['PassengerId',
 'Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

In [8]:
feat_cols = my_data.select(['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked'])

In [9]:
feat_cols.show()

+--------+------+------+----+-----+-----+-------+--------+
Survived|Pclass| Sex| Age|SibSp|Parch| Fare|Embarked|
+--------+------+------+----+-----+-----+-------+--------+
 0| 3| male|22.0| 1| 0| 7.25| S|
 1| 1|female|38.0| 1| 0|71.2833| C|
 1| 3|female|26.0| 0| 0| 7.925| S|
 1| 1|female|35.0| 1| 0| 53.1| S|
 0| 3| male|35.0| 0| 0| 8.05| S|
 0| 3| male|null| 0| 0| 8.4583| Q|
 0| 1| male|54.0| 0| 0|51.8625| S|
 0| 3| male| 2.0| 3| 1| 21.075| S|
 1| 3|female|27.0| 0| 2|11.1333| S|
 1| 2|female|14.0| 1| 0|30.0708| C|
 1| 3|female| 4.0| 1| 1| 16.7| S|
 1| 1|female|58.0| 0| 0| 26.55| S|
 0| 3| male|20.0| 0| 0| 8.05| S|
 0| 3| male|39.0| 1| 5| 31.275| S|
 0| 3|female|14.0| 0| 0| 7.8542| S|
 1| 2|female|55.0| 0| 0| 16.0| S|
 0| 3| male| 2.0| 4| 1| 29.125| Q|
 1| 2| male|null| 0| 0| 13.0| S|
 0| 3|female|31.0| 1| 0| 18.0| S|
 1| 3|female|null| 0| 0| 7.225| C|
+--------+------+------+----+-----+-----+-------+--------+
only showing top 20 rows

In [10]:
final_data = feat_cols.na.drop()

In [11]:
final_data.show()

+--------+------+------+----+-----+-----+-------+--------+
Survived|Pclass| Sex| Age|SibSp|Parch| Fare|Embarked|
+--------+------+------+----+-----+-----+-------+--------+
 0| 3| male|22.0| 1| 0| 7.25| S|
 1| 1|female|38.0| 1| 0|71.2833| C|
 1| 3|female|26.0| 0| 0| 7.925| S|
 1| 1|female|35.0| 1| 0| 53.1| S|
 0| 3| male|35.0| 0| 0| 8.05| S|
 0| 1| male|54.0| 0| 0|51.8625| S|
 0| 3| male| 2.0| 3| 1| 21.075| S|
 1| 3|female|27.0| 0| 2|11.1333| S|
 1| 2|female|14.0| 1| 0|30.0708| C|
 1| 3|female| 4.0| 1| 1| 16.7| S|
 1| 1|female|58.0| 0| 0| 26.55| S|
 0| 3| male|20.0| 0| 0| 8.05| S|
 0| 3| male|39.0| 1| 5| 31.275| S|
 0| 3|female|14.0| 0| 0| 7.8542| S|
 1| 2|female|55.0| 0| 0| 16.0| S|
 0| 3| male| 2.0| 4| 1| 29.125| Q|
 0| 3|female|31.0| 1| 0| 18.0| S|
 0| 2| male|35.0| 0| 0| 26.0| S|
 1| 2| male|34.0| 0| 0| 13.0| S|
 1| 3|female|15.0| 0| 0| 8.0292| Q|
+--------+------+------+----+-----+-----+-------+--------+
only showing top 20 rows

In [12]:
# categorical columns transform
from pyspark.ml.feature import (VectorAssembler, VectorIndexer,
                                OneHotEncoder, StringIndexer)

In [13]:
gender_indexer = StringIndexer(inputCol= 'Sex', outputCol= 'SexIndex')

In [14]:
gender_encoder = OneHotEncoder(inputCol= 'SexIndex', outputCol= 'SexVec')

In [15]:
embark_indexer = StringIndexer(inputCol= 'Embarked', outputCol= 'EmbarkedIndex')
embark_encoder = OneHotEncoder(inputCol= 'EmbarkedIndex', outputCol= 'EmbarkedVec')


In [16]:
assembler = VectorAssembler(inputCols= ['Pclass', 'SexVec', 'Age', 'SibSp', 'Parch', 'Fare', 'EmbarkedVec'],
                           outputCol = 'features')

In [17]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline

In [18]:
log_titanic = LogisticRegression(featuresCol= 'features', 
                                labelCol = 'Survived')

In [19]:
pipeline = Pipeline(stages = [gender_indexer,
                             embark_indexer,
                             gender_encoder,
                             embark_encoder,
                             assembler,
                             log_titanic])

In [20]:
train_data, test_data = final_data.randomSplit([0.7, 0.3])

In [21]:
fit_model = pipeline.fit(train_data)

In [22]:
results = fit_model.transform(test_data)

In [23]:
results.select(['Survived', 'prediction']).show()

+--------+----------+
Survived|prediction|
+--------+----------+
 0| 1.0|
 0| 1.0|
 0| 1.0|
 0| 1.0|
 0| 1.0|
 0| 1.0|
 0| 0.0|
 0| 1.0|
 0| 0.0|
 0| 0.0|
 0| 1.0|
 0| 0.0|
 0| 1.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
+--------+----------+
only showing top 20 rows

In [24]:
#evaluator
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [25]:
my_eval = BinaryClassificationEvaluator(rawPredictionCol= 'prediction', labelCol = 'Survived')

In [26]:
AUC = my_eval.evaluate(results)
AUC

Out[ 41 ]: 0.7414981617647058